# Предсказание покупки следующего курса

### Загрузим необходимые библиотеки

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

### Загружаем датасеты

In [400]:
df = pd.read_csv("./content/train_dataset_train.csv")
dft = pd.read_csv("./content/test_dataset_test.csv")

### и объединяем их

In [401]:
len_df = len(df)
dfres = pd.concat([df, dft], ignore_index=True)
#len(dfres)
#dfres = dfres.fillna(0)
#dfres.info(verbose=True)

In [402]:
dfres = dfres.fillna(-1)

### Рассмотрим датасеты по ближе

In [403]:
df.head(3)

,id,age_indicator,month_id,student_id,program_id,carts_created_at,spent_time_total,spent_time_to_complete_hw,completed_hw,failed_hw,...,p_total_calls,p_was_conversations,p_total_duration,support_feedback_avg,feedback_avg_d1,feedback_avg_d2,feedback_avg_d3,feedback_avg_d4,feedback_avg_d5,target
0,15182,32.0,9/30/2020,6694527,1469,8/26/2020,163.0,0.0,1.0,0.0,...,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,0
1,89385,NaN,6/30/2021,6712877,1392,8/5/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,47931,NaN,2/28/2021,6659444,376,6/20/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


```
 0   id                         200000 non-null  int64   # уникальное значение каждой строк
 1   age_indicator              159123 non-null  float64 # возраст в условных единицах
 2   month_id                   200000 non-null  object  # последний день месяца среза
 3   student_id                 200000 non-null  int64   # id студента  
 4   program_id                 200000 non-null  int64   # id продукта
 5   carts_created_at           200000 non-null  object  # дата создания заявки
 6   spent_time_total           86309 non-null   float64 # время за учебой в условных единицах
 7   spent_time_to_complete_hw  42467 non-null   float64 # время, потраченное на выполнение дз
 8   completed_hw               97599 non-null   float64 # кол-во выполненных дз
 9   failed_hw                  97599 non-null   float64 # кол-во проваленных дз 
 10  reworked_hw                97599 non-null   float64 # кол-во переделанных дз 
 11  interacted_hw              97599 non-null   float64 # кол-во дз, которыми занимался студент в этом месяце
 12  avg_hw_mark                44496 non-null   float64 # средняя оценка за дз
 13  test_with_good_mark        97599 non-null   float64 # кол-во тестов с удовлетворительной оценкой
 14  test_with_great_mark       97599 non-null   float64 # кол-во тестов с отличной оценкой
 15  webinars                   97599 non-null   float64 # кол-во просмотренных вебинаров
 16  avg_quiz_result            16259 non-null   float64 # средняя оценка за квизы
 17  notes                      97599 non-null   float64 # кол-во оставленных заметок
 18  hw_leader                  97599 non-null   float64 # был лидером в групповых заданиях
 19  lessons                    97599 non-null   float64 # кол-во уроков, которые открывал пользователь
 20  activity                   97599 non-null   float64 # общая активность студента в личном кабинете
 21  bought_d1                  200000 non-null  int64   # кол-во прошлых покупок в направлении 1
 22  bought_d2                  200000 non-null  int64   
 23  bought_d3                  200000 non-null  int64  
 24  bought_d4                  200000 non-null  int64  
 25  bought_d5                  200000 non-null  int64  
 26  bought_avg_duration        10359 non-null   float64
 27  payment_type               200000 non-null  int64   # тип оплаты
 28  promo                      200000 non-null  object  # участвовал ли в любой промокампании
 29  price                      200000 non-null  float64 # цена программы
 30  communication_type         198595 non-null  object  # тип создания заявки (ОП или сам через сайт)
 31  auto_payment               200000 non-null  int64   # автопокупка (1 или 0)
 32  ABC                        200000 non-null  object  # сегмент клиента по готовности купить
 33  city                       13592 non-null   object  # город пользователя
 34  country                    12944 non-null   object  # страна пользователя 
 35  gender                     199940 non-null  float64 # пол
 36  speed_recall               21880 non-null   float64 # скорость перезвона клиенту (1 - быстро, 7 - долго)
 37  os                         77374 non-null   object  # операционная система пользователя
 38  browser                    77374 non-null   object  # название браузера 
 39  platform                   77361 non-null   object  # платформа
 40  m_avg_talk_duration        51537 non-null   float64 # средняя длительность разговора
 41  m_avg_duration             45764 non-null   float64 # средняя длительность звонка 
 42  m_missed_calls             51537 non-null   float64 # кол-во пропущенных звонков
 43  m_total_calls              51537 non-null   float64 # всего звонков
 44  m_was_conversations        51537 non-null   float64 # кол-во звонков, когда состоялся диалог
 45  m_total_duration           51537 non-null   float64 # длительность всего
 46  p_avg_talk_duration        16453 non-null   float64
 47  p_avg_duration             13488 non-null   float64
 48  p_missed_calls             16453 non-null   float64
 49  p_total_calls              16453 non-null   float64
 50  p_was_conversations        16453 non-null   float64
 51  p_total_duration           16453 non-null   float64
 52  support_feedback_avg       37557 non-null   float64 # средняя удовлетворенность службой поддержки за год
 53  feedback_avg_d1            48496 non-null   float64 # средний балл по курсам направления 1
 54  feedback_avg_d2            14805 non-null   float64
 55  feedback_avg_d3            17778 non-null   float64
 56  feedback_avg_d4            25076 non-null   float64
 57  feedback_avg_d5            15733 non-null   float64
 58  target                     200000 non-null  int64  
``` 

### Приспособы для one_hot_encoding

In [404]:
def convert_to_dict(df):
    res = dict()
    i = 0
    for el in df:
        if el == -1:
            continue
        if el not in res:
            res[el] = i
            i += 1
    return res

In [405]:
def simple_one_hot_encoding(df, col_name):
    codes = convert_to_dict(df[col_name])
        
    new_columns = {}
    for i in range(len(codes)):
        new_columns[col_name+str(i)] = [0 for i in range(len(df))]
    new_columns = pd.DataFrame(new_columns, index=df.index)
    new_columns = new_columns.astype(np.int8)
    df = pd.concat([df, new_columns], axis=1)

    for i in range(len(df)):
        code = df.iloc[i, df.columns.get_loc(col_name)]
        if code != -1:
            ccode = codes[code]
            df.iloc[i, df.columns.get_loc(col_name+str(ccode))] = 1
    return df, codes

In [406]:
def digital_convert_to_dict(vmax):
    res = dict()
    
    i = 0
    j = 0
    while i <= vmax:
        if i < 10:
            res[j] = i
            i += 1
            j += 1
        elif i < 100:
            res[j] = i
            i += 10
            j += 1
        elif i < 1000:
            res[j] = i
            i += 100
            j += 1
        elif i < 10000:
            res[j] = i
            i += 1000
            j += 1
        elif i < 100000:
            res[j] = i
            i += 1000
            j += 1
        elif i < 1000000:
            res[j] = i
            i += 10000
            j += 1
    return res

In [407]:
def digital_conv(code, codes):
    if code == -1:
        return None
    prev_c = 0
    for c in codes:
        if code >= codes[c]:
            prev_c = c
        else:
            break
    return prev_c

In [408]:
def digital_one_hot_encoding(df, col_name):
    
    vmax = df[col_name].max()
    
    codes = digital_convert_to_dict(vmax)
        
    new_columns = {}
    for code in range(len(codes)):
        new_columns[col_name+str(code)] = [0 for i in range(len(df))]
    new_columns = pd.DataFrame(new_columns, index=df.index)
    new_columns = new_columns.astype(np.int8)
    df = pd.concat([df, new_columns], axis=1)

    for i in range(len(df)):
        code = df.iloc[i, df.columns.get_loc(col_name)]
        ccode = digital_conv(code, codes)
        if ccode != None:
            df.iloc[i, df.columns.get_loc(col_name+str(ccode))] = 1
    return df, codes

### анализируем данные

In [409]:
# кандидат на one_hoting_encode
d = dfres['age_indicator'].value_counts()
l = len(d)
rmax = dfres['age_indicator'].max()
rmin = dfres['age_indicator'].min()
d, l, rmax, rmin

(-1.0     58326
  33.0     5767
  25.0     5759
  35.0     5742
  15.0     5742
  21.0     5739
  30.0     5737
  39.0     5732
  2.0      5730
  10.0     5727
  13.0     5726
  4.0      5725
  9.0      5724
  8.0      5724
  28.0     5724
  36.0     5723
  38.0     5722
  1.0      5721
  16.0     5715
  22.0     5711
  31.0     5711
  11.0     5711
  29.0     5709
  32.0     5703
  6.0      5701
  27.0     5701
  12.0     5700
  23.0     5699
  7.0      5699
  19.0     5698
  37.0     5689
  3.0      5688
  17.0     5686
  24.0     5676
  26.0     5669
  14.0     5664
  20.0     5652
  34.0     5626
  5.0      5620
  18.0     5232
  40.0     4547
 Name: age_indicator, dtype: int64,
 41,
 40.0,
 -1.0)

In [410]:
dfres, codes_age_indicator = digital_one_hot_encoding(dfres, 'age_indicator')
dfres = dfres.drop('age_indicator', axis=1)

In [411]:
# кандидат на one_hoting_encode
d = dfres['month_id'].value_counts()
l = len(d)
rmax = dfres['month_id'].max()
rmin = dfres['month_id'].min()
d, l, rmax, rmin

(11/30/2021    34088
 10/31/2021    31019
 9/30/2021     28439
 8/31/2021     25913
 7/31/2021     23513
 6/30/2021     21552
 5/31/2021     19651
 4/30/2021     17936
 3/31/2021     16170
 2/28/2021     14298
 1/31/2021     12680
 12/31/2020    10857
 11/30/2020     8986
 10/31/2020     7276
 9/30/2020      5792
 8/31/2020      3844
 7/31/2020      2211
 6/30/2020       772
 Name: month_id, dtype: int64,
 18,
 '9/30/2021',
 '1/31/2021')

In [412]:
dfres, codes_month_id = simple_one_hot_encoding(dfres, 'month_id')
dfres = dfres.drop('month_id', axis=1)

In [413]:
# кандидат на one_hoting_encode
d = dfres['program_id'].value_counts()
l = len(d)
rmax = dfres['program_id'].max()
rmin = dfres['program_id'].min()
d, l, rmax, rmin

(789     11492
 1360     8414
 1635     7974
 1609     7492
 211      7401
         ...  
 2611        2
 2699        1
 2714        1
 2235        1
 2514        1
 Name: program_id, Length: 269, dtype: int64,
 269,
 2714,
 12)

In [414]:
dfres, codes_program_id = simple_one_hot_encoding(dfres, 'program_id')
dfres = dfres.drop('program_id', axis=1)

In [415]:
# считаем, что дата создания заявки не очень важна
d = dfres['carts_created_at'].value_counts()
l = len(d)
rmax = dfres['carts_created_at'].max()
rmin = dfres['carts_created_at'].min()
d, l, rmax, rmin

(12/30/2020    4119
 8/24/2020     2568
 8/27/2020     2502
 12/31/2020    2481
 8/31/2020     2410
               ... 
 11/16/2021      49
 6/6/2020        36
 6/8/2020        36
 6/9/2020        18
 6/5/2020        18
 Name: carts_created_at, Length: 447, dtype: int64,
 447,
 '9/9/2021',
 '1/1/2021')

In [416]:
dfres, codes_carts_created_at = simple_one_hot_encoding(dfres, 'carts_created_at')
dfres = dfres.drop('carts_created_at', axis=1)

In [417]:
d = dfres['spent_time_total'].value_counts()
l = len(d)
rmax = dfres['spent_time_total'].max()
rmin = dfres['spent_time_total'].min()
d, l, rmax, rmin

(-1.0       162001
  0.0        11656
  1.0         3182
  2.0         2740
  3.0         2359
             ...  
  761.0          1
  1148.0         1
  1229.0         1
  1383.0         1
  917.0          1
 Name: spent_time_total, Length: 1018, dtype: int64,
 1018,
 3111.0,
 -1.0)

In [418]:
dfres, codes_spent_time_total = digital_one_hot_encoding(dfres, 'spent_time_total')
dfres = dfres.drop('spent_time_total', axis=1)

In [419]:
# игнорим
d = dfres['spent_time_to_complete_hw'].value_counts()
l = len(d)
rmax = dfres['spent_time_to_complete_hw'].max()
rmin = dfres['spent_time_to_complete_hw'].min()
d, l, rmax, rmin

(-1.0    224487
  0.0     60510
 Name: spent_time_to_complete_hw, dtype: int64,
 2,
 0.0,
 -1.0)

In [420]:
#dfres, codes_program_id = digital_one_hot_encoding(dfres, 'spent_time_to_complete_hw')
dfres = dfres.drop('spent_time_to_complete_hw', axis=1)

In [421]:
# кандидат на one_hoting_encode
d = dfres['completed_hw'].value_counts()
l = len(d)
rmax = dfres['completed_hw'].max()
rmin = dfres['completed_hw'].min()
d, l, rmax, rmin

(-1.0     145917
  0.0      75073
  1.0      20886
  2.0      12811
  3.0       9534
  4.0       7733
  5.0       5404
  6.0       3788
  7.0       2124
  8.0       1060
  9.0        415
  10.0       155
  11.0        46
  12.0        19
  13.0        15
  14.0         6
  15.0         3
  17.0         2
  16.0         2
  25.0         2
  34.0         1
  23.0         1
 Name: completed_hw, dtype: int64,
 22,
 34.0,
 -1.0)

In [422]:
dfres, codes_completed_hw = digital_one_hot_encoding(dfres, 'completed_hw')
dfres = dfres.drop('completed_hw', axis=1)

In [423]:
# разобьем на шаги по 10
d = dfres['failed_hw'].value_counts()
l = len(d)
rmax = dfres['failed_hw'].max()
rmin = dfres['failed_hw'].min()
d, l, rmax, rmin

(-1.0      145917
  0.0      138746
  1.0          74
  2.0          29
  8.0          19
  3.0          17
  4.0          15
  12.0         14
  6.0          14
  15.0         13
  5.0          12
  10.0         11
  7.0          10
  9.0          10
  20.0          9
  11.0          9
  16.0          8
  19.0          7
  18.0          7
  17.0          6
  14.0          5
  21.0          4
  23.0          4
  26.0          4
  13.0          3
  30.0          3
  41.0          3
  33.0          3
  25.0          3
  28.0          3
  31.0          2
  36.0          2
  27.0          2
  102.0         1
  24.0          1
  60.0          1
  57.0          1
  29.0          1
  22.0          1
  38.0          1
  37.0          1
  43.0          1
 Name: failed_hw, dtype: int64,
 42,
 102.0,
 -1.0)

In [424]:
dfres, codes_failed_hw = digital_one_hot_encoding(dfres, 'failed_hw')
dfres = dfres.drop('failed_hw', axis=1)

In [425]:
# разобьем на шаги по 10
d = dfres['reworked_hw'].value_counts()
l = len(d)
rmax = dfres['reworked_hw'].max()
rmin = dfres['reworked_hw'].min()
d, l, rmax, rmin

(-1.0      145917
  0.0      107712
  11.0       1317
  10.0       1298
  13.0       1282
            ...  
  349.0         1
  136.0         1
  151.0         1
  320.0         1
  184.0         1
 Name: reworked_hw, Length: 227, dtype: int64,
 227,
 721.0,
 -1.0)

In [426]:
dfres, codes_reworked_hw = digital_one_hot_encoding(dfres, 'reworked_hw')
dfres = dfres.drop('reworked_hw', axis=1)

In [427]:
#с шагом 1
d = dfres['interacted_hw'].value_counts()
l = len(d)
rmax = dfres['interacted_hw'].max()
rmin = dfres['interacted_hw'].min()
d, l, rmax, rmin

(-1.0     145917
  2.0      71208
  3.0      21828
  4.0      13502
  5.0      10063
  6.0       8140
  7.0       5821
  8.0       4052
  9.0       2377
  10.0      1241
  11.0       501
  12.0       193
  13.0        61
  14.0        41
  15.0        22
  16.0        10
  17.0         5
  18.0         3
  19.0         3
  21.0         2
  27.0         2
  23.0         2
  38.0         1
  32.0         1
  26.0         1
 Name: interacted_hw, dtype: int64,
 25,
 38.0,
 -1.0)

In [428]:
dfres, codes_interacted_hw = digital_one_hot_encoding(dfres, 'interacted_hw')
dfres = dfres.drop('interacted_hw', axis=1)

In [429]:
#c шагом 10, max=100
d = dfres['avg_hw_mark'].value_counts()
l = len(d)
rmax = dfres['avg_hw_mark'].max()
rmin = dfres['avg_hw_mark'].min()
d, l, rmax, rmin

(-1.000000      221592
  100.000000     62689
  0.000000         385
  50.000000         50
  66.666667         25
                 ...  
  93.103448          1
  23.529412          1
  78.787879          1
  82.352941          1
  78.947368          1
 Name: avg_hw_mark, Length: 82, dtype: int64,
 82,
 100.0,
 -1.0)

In [430]:
dfres, codes_avg_hw_mark = digital_one_hot_encoding(dfres, 'avg_hw_mark')
dfres = dfres.drop('avg_hw_mark', axis=1)

In [431]:
# с шагом 10
d = dfres['test_with_good_mark'].value_counts()
l = len(d)
rmax = dfres['test_with_good_mark'].max()
rmin = dfres['test_with_good_mark'].min()
d, l, rmax, rmin

(-1.0      145917
  0.0      103933
  4.0       11611
  8.0        7941
  12.0       4292
            ...  
  160.0         1
  208.0         1
  286.0         1
  184.0         1
  216.0         1
 Name: test_with_good_mark, Length: 93, dtype: int64,
 93,
 418.0,
 -1.0)

In [432]:
dfres, codes_test_with_good_mark = digital_one_hot_encoding(dfres, 'test_with_good_mark')
dfres = dfres.drop('test_with_good_mark', axis=1)

In [433]:
# с шагом 10
d = dfres['test_with_great_mark'].value_counts()
l = len(d)
rmax = dfres['test_with_great_mark'].max()
rmin = dfres['test_with_great_mark'].min()
d, l, rmax, rmin

(-1.0      145917
  0.0      106285
  4.0        9131
  6.0        4277
  8.0        3937
            ...  
  110.0         1
  104.0         1
  120.0         1
  208.0         1
  130.0         1
 Name: test_with_great_mark, Length: 63, dtype: int64,
 63,
 208.0,
 -1.0)

In [434]:
dfres, codes_test_with_great_mark = digital_one_hot_encoding(dfres, 'test_with_great_mark')
dfres = dfres.drop('test_with_great_mark', axis=1)

In [435]:
# с шагом 10
d = dfres['webinars'].value_counts()
l = len(d)
rmax = dfres['webinars'].max()
rmin = dfres['webinars'].min()
d, l, rmax, rmin

(-1.0      145917
  0.0       77431
  1.0       13424
  2.0        9447
  3.0        7029
            ...  
  102.0         1
  55.0          1
  67.0          1
  56.0          1
  65.0          1
 Name: webinars, Length: 71, dtype: int64,
 71,
 242.0,
 -1.0)

In [436]:
dfres, codes_webinars = digital_one_hot_encoding(dfres, 'webinars')
dfres = dfres.drop('webinars', axis=1)

In [437]:
d = dfres['avg_quiz_result'].value_counts()
l = len(d)
rmax = dfres['avg_quiz_result'].max()
rmin = dfres['avg_quiz_result'].min()
d, l, rmax, rmin

(-1.000000     261784
  97.000000      7814
  87.300000      1295
  77.600000      1160
  89.538462       520
                ...  
  89.011765         1
  66.103704         1
  12.933333         1
  89.533294         1
  66.511775         1
 Name: avg_quiz_result, Length: 3394, dtype: int64,
 3394,
 97.0,
 -1.0)

In [438]:
dfres, codes_avg_quiz_result = digital_one_hot_encoding(dfres, 'avg_quiz_result')
dfres = dfres.drop('avg_quiz_result', axis=1)

In [439]:
# с шагом 100
d = dfres['notes'].value_counts()
l = len(d)
rmax = dfres['notes'].max()
rmin = dfres['notes'].min()
d, l, rmax, rmin

(-1.0        145917
  0.0        129303
  105.0         614
  84.0          500
  21.0          429
              ...  
  8883.0          1
  6951.0          1
  6363.0          1
  8295.0          1
  11109.0         1
 Name: notes, Length: 349, dtype: int64,
 349,
 13377.0,
 -1.0)

In [440]:
dfres, codes_notes = digital_one_hot_encoding(dfres, 'notes')
dfres = dfres.drop('notes', axis=1)

In [441]:
# С шагом 10
d = dfres['hw_leader'].value_counts()
l = len(d)
rmax = dfres['hw_leader'].max()
rmin = dfres['hw_leader'].min()
d, l, rmax, rmin

(-1.0      145917
  0.0      138015
  1.0          92
  8.0          73
  6.0          70
  5.0          68
  7.0          67
  4.0          67
  9.0          66
  10.0         65
  12.0         47
  2.0          44
  11.0         40
  13.0         38
  14.0         36
  3.0          32
  16.0         29
  15.0         29
  17.0         21
  18.0         21
  20.0         14
  22.0         14
  28.0         13
  19.0         13
  21.0         12
  24.0         11
  25.0          9
  31.0          7
  30.0          6
  29.0          6
  23.0          6
  26.0          6
  27.0          5
  43.0          4
  32.0          3
  33.0          3
  37.0          3
  41.0          3
  35.0          2
  51.0          2
  44.0          2
  42.0          2
  34.0          2
  54.0          1
  50.0          1
  48.0          1
  49.0          1
  138.0         1
  80.0          1
  45.0          1
  36.0          1
  39.0          1
  66.0          1
  93.0          1
  57.0          1
 Name: hw_

In [442]:
dfres, codes_hw_leader = digital_one_hot_encoding(dfres, 'hw_leader')
dfres = dfres.drop('hw_leader', axis=1)

In [443]:
d = dfres['lessons'].value_counts()
l = len(d)
rmax = dfres['lessons'].max()
rmin = dfres['lessons'].min()
d, l, rmax, rmin

(-1.0     145917
  1.0      14661
  2.0       9565
  9.0       8927
  8.0       8767
           ...  
  50.0         1
  75.0         1
  61.0         1
  83.0         1
  53.0         1
 Name: lessons, Length: 66, dtype: int64,
 66,
 85.0,
 -1.0)

In [444]:
dfres, codes_lessons = digital_one_hot_encoding(dfres, 'lessons')
dfres = dfres.drop('lessons', axis=1)

In [445]:
d = dfres['activity'].value_counts()
l = len(d)
rmax = dfres['activity'].max()
rmin = dfres['activity'].min()
d, l, rmax, rmin

(-1.0      145917
  0.0       20852
  1.0        9566
  2.0        6967
  3.0        5728
            ...  
  148.0         1
  131.0         1
  137.0         1
  133.0         1
  121.0         1
 Name: activity, Length: 148, dtype: int64,
 148,
 196.0,
 -1.0)

In [446]:
dfres, codes_activity = digital_one_hot_encoding(dfres, 'activity')
dfres = dfres.drop('activity', axis=1)

In [447]:
d = dfres['bought_d1'].value_counts()
l = len(d)
rmax = dfres['bought_d1'].max()
rmin = dfres['bought_d1'].min()
d, l, rmax, rmin

(0     277419
 1       6467
 3        782
 4        211
 6         79
 7         21
 9          6
 10         6
 12         5
 13         1
 Name: bought_d1, dtype: int64,
 10,
 13,
 0)

In [448]:
dfres, codes_bought_d1 = digital_one_hot_encoding(dfres, 'bought_d1')
dfres = dfres.drop('bought_d1', axis=1)

In [449]:
d = dfres['bought_d2'].value_counts()
l = len(d)
rmax = dfres['bought_d2'].max()
rmin = dfres['bought_d2'].min()
d, l, rmax, rmin

(0    283110
 1      1731
 3       126
 4        30
 Name: bought_d2, dtype: int64,
 4,
 4,
 0)

In [450]:
dfres, codes_bought_d2 = digital_one_hot_encoding(dfres, 'bought_d2')
dfres = dfres.drop('bought_d2', axis=1)

In [451]:
d = dfres['bought_d3'].value_counts()
l = len(d)
rmax = dfres['bought_d3'].max()
rmin = dfres['bought_d3'].min()
d, l, rmax, rmin

(0    283714
 1      1200
 4        25
 3        25
 7        20
 6        13
 Name: bought_d3, dtype: int64,
 6,
 7,
 0)

In [452]:
dfres, codes_bought_d3 = digital_one_hot_encoding(dfres, 'bought_d3')
dfres = dfres.drop('bought_d3', axis=1)

In [453]:
d = dfres['bought_d4'].value_counts()
l = len(d)
rmax = dfres['bought_d4'].max()
rmin = dfres['bought_d4'].min()
d, l, rmax, rmin

(0    283157
 1      1782
 3        52
 4         6
 Name: bought_d4, dtype: int64,
 4,
 4,
 0)

In [454]:
dfres, codes_bought_d4 = digital_one_hot_encoding(dfres, 'bought_d4')
dfres = dfres.drop('bought_d4', axis=1)

In [455]:
d = dfres['bought_d5'].value_counts()
l = len(d)
rmax = dfres['bought_d5'].max()
rmin = dfres['bought_d5'].min()
d, l, rmax, rmin

(0    281886
 1      2789
 3       256
 4        54
 6        12
 Name: bought_d5, dtype: int64,
 5,
 6,
 0)

In [456]:
dfres, codes_bought_d5 = digital_one_hot_encoding(dfres, 'bought_d5')
dfres = dfres.drop('bought_d5', axis=1)

In [457]:
d = dfres['bought_avg_duration'].value_counts()
l = len(d)
rmax = dfres['bought_avg_duration'].max()
rmin = dfres['bought_avg_duration'].min()
d, l, rmax, rmin

(-1.0      270361
  16.0        818
  24.0        661
  18.0        608
  32.0        516
            ...  
  144.0         2
  52.0          2
  145.0         1
  129.0         1
  142.0         1
 Name: bought_avg_duration, Length: 160, dtype: int64,
 160,
 186.0,
 -1.0)

In [458]:
dfres, codes_bought_avg_duration = digital_one_hot_encoding(dfres, 'bought_avg_duration')
dfres = dfres.drop('bought_avg_duration', axis=1)

In [459]:
d = dfres['payment_type'].value_counts()
l = len(d)
rmax = dfres['payment_type'].max()
rmin = dfres['payment_type'].min()
d, l, rmax, rmin

(1    257228
 2     27769
 Name: payment_type, dtype: int64,
 2,
 2,
 1)

In [460]:
dfres, codes_payment_type = simple_one_hot_encoding(dfres, 'payment_type')
dfres = dfres.drop('payment_type', axis=1)

In [461]:
d = dfres['promo'].value_counts()
l = len(d)
rmax = dfres['promo'].max()
rmin = dfres['promo'].min()
d, l, rmax, rmin

(-    197607
 +     87390
 Name: promo, dtype: int64,
 2,
 '-',
 '+')

In [462]:
dfres, codes_promo = simple_one_hot_encoding(dfres, 'promo')
dfres = dfres.drop('promo', axis=1)

In [463]:
d = dfres['price'].value_counts()
l = len(d)
rmax = dfres['price'].max()
rmin = dfres['price'].min()
d, l, rmax, rmin

(20042.95930    4354
 24060.81720    2093
 30855.91320    2060
 22260.63222    2044
 29311.57320    1863
                ... 
 69592.48412       1
 32835.47977       1
 17992.81433       1
 56486.83895       1
 35676.82790       1
 Name: price, Length: 7484, dtype: int64,
 7484,
 125643.3649,
 0.0)

In [464]:
dfres, codes_price = digital_one_hot_encoding(dfres, 'price')
dfres = dfres.drop('price', axis=1)

In [465]:
d = dfres['communication_type'].value_counts()
l = len(d)
#rmax = dfres['communication_type'].max()
#rmin = dfres['communication_type'].min()
d, l, #rmax, rmin

(order    172219
 web       70768
 phone     39969
 -1         2041
 Name: communication_type, dtype: int64,
 4)

In [466]:
dfres, codes_communication_type = simple_one_hot_encoding(dfres, 'communication_type')
dfres = dfres.drop('communication_type', axis=1)

In [467]:
d = dfres['auto_payment'].value_counts()
l = len(d)
d, l, #rmax, rmin

(0    162505
 1    122492
 Name: auto_payment, dtype: int64,
 2)

In [468]:
dfres, codes_auto_payment = simple_one_hot_encoding(dfres, 'auto_payment')
dfres = dfres.drop('auto_payment', axis=1)

In [469]:
d = dfres['ABC'].value_counts()
l = len(d)
d, l

(D    142848
 A     65616
 B     60685
 C     15848
 Name: ABC, dtype: int64,
 4)

In [470]:
dfres, codes_ABC = simple_one_hot_encoding(dfres, 'ABC')
dfres = dfres.drop('ABC', axis=1)

In [471]:
d = dfres['city'].value_counts()
l = len(d)
d, l

(-1                  265736
 Москва                8460
 Санкт-Петербург       2383
 Екатеринбург           395
 Нижний Новгород        313
                      ...  
 Almere                   1
 Великий Новгород         1
 Cannes                   1
 Дербент                  1
 Piestany                 1
 Name: city, Length: 280, dtype: int64,
 280)

In [472]:
dfres, codes_city = simple_one_hot_encoding(dfres, 'city')
dfres = dfres.drop('city', axis=1)

In [473]:
d = dfres['country'].value_counts()
l = len(d)
#rmax = dfres['price'].max()
#rmin = dfres['price'].min()
d, l, #rmax, rmin

(-1                       266630
 Россия                    16679
 Беларусь                    286
 РФ                          282
 Казахстан                   227
 Украина                     195
 Кыргызстан                   64
 Росссия                      64
 Кипр                         58
 Латвия                       48
 Германия                     37
 Китай                        32
 Франция                      32
 Чешская Республика           31
 Бельгия                      27
 Швейцария                    26
 Испания                      25
 <span>Россия</span>          24
 США                          22
 Польша                       19
 Латвия                       17
 Люксембург                   16
 Великобритания               16
 Литва                        15
 Канада                       14
 Турция                       13
 Чили                         13
 Израиль                      12
 Италия                       10
 Кыргызская Республика        10
 Нидерланд

In [474]:
dfres, codes_country = simple_one_hot_encoding(dfres, 'country')
dfres = dfres.drop('country', axis=1)

In [475]:
d = dfres['gender'].value_counts()
l = len(d)
rmax = dfres['gender'].max()
rmin = dfres['gender'].min()
d, l, #rmax, rmin

( 0.0    124882
  1.0     98534
  2.0     61502
 -1.0        79
 Name: gender, dtype: int64,
 4)

In [476]:
dfres, codes_gender = simple_one_hot_encoding(dfres, 'gender')
dfres = dfres.drop('gender', axis=1)

In [477]:
d = dfres['speed_recall'].value_counts()
l = len(d)
rmax = dfres['speed_recall'].max()
rmin = dfres['speed_recall'].min()
d, l, rmax, rmin

(-1.0    253834
  1.0     28032
  4.0       829
  3.0       712
  2.0       650
  5.0       564
  6.0       278
  7.0        98
 Name: speed_recall, dtype: int64,
 8,
 7.0,
 -1.0)

In [478]:
dfres, codes_speed_recall = digital_one_hot_encoding(dfres, 'speed_recall')
dfres = dfres.drop('speed_recall', axis=1)

In [479]:
d = dfres['os'].value_counts()
l = len(d)
d, l

(-1           174597
 Windows       52704
 iOS           22323
 Android       20040
 Mac OS X      14430
 Linux           641
 Ubuntu          220
 Chrome OS        21
 Fedora           21
 Name: os, dtype: int64,
 9)

In [480]:
dfres, codes_os = simple_one_hot_encoding(dfres, 'os')
dfres = dfres.drop('os', axis=1)

In [481]:
d = dfres['browser'].value_counts()
l = len(d)
d, l

(-1                            174597
 Chrome                         43660
 Mobile Safari                  17585
 Chrome Mobile                  11742
 Yandex Browser                 11689
 Safari                          7383
 Firefox                         3851
 Opera                           3420
 Edge                            2674
 Instagram                       2324
 Samsung Internet                1808
 Chrome Mobile iOS               1431
 MiuiBrowser                      855
 Google                           546
 Facebook                         416
 Opera Mobile                     337
 Chrome Mobile WebView            232
 Firefox Mobile                   171
 YandexSearch                      73
 HeadlessChrome                    48
 Mobile Safari UI/WKWebView        43
 Firefox iOS                       37
 Edge Mobile                       30
 Chromium                          17
 UC Browser                        11
 Iron                               9
 Android    

In [482]:
dfres, codes_browser = simple_one_hot_encoding(dfres, 'browser')
dfres = dfres.drop('browser', axis=1)

In [483]:
d = dfres['platform'].value_counts()
l = len(d)
d, l

(-1        174618
 pc         68016
 mobile     41690
 tablet       673
 Name: platform, dtype: int64,
 4)

In [484]:
dfres, codes_platform = simple_one_hot_encoding(dfres, 'platform')
dfres = dfres.drop('platform', axis=1)

In [485]:
d = dfres['m_avg_talk_duration'].value_counts()
l = len(d)
rmax = dfres['m_avg_talk_duration'].max()
rmin = dfres['m_avg_talk_duration'].min()
d, l, rmax, rmin

(-1.0000      211552
  0.0000        8424
  3.0000         381
  5.0000         316
  6.0000         305
               ...  
  846.5000         1
  236.7500         1
  486.9942         1
  287.5000         1
  126.8000         1
 Name: m_avg_talk_duration, Length: 5368, dtype: int64,
 5368,
 3658.7811,
 -1.0)

In [486]:
dfres, codes_m_avg_talk_duration = digital_one_hot_encoding(dfres, 'm_avg_talk_duration')
dfres = dfres.drop('m_avg_talk_duration', axis=1)

In [487]:
d = dfres['p_avg_talk_duration'].value_counts()
l = len(d)
rmax = dfres['p_avg_talk_duration'].max()
rmin = dfres['p_avg_talk_duration'].min()
d, l, rmax, rmin

(-1.000000      261415
  0.000000        4270
  38.000000        286
  42.000000        242
  26.000000        223
                 ...  
  67.989450          1
  164.439600         1
  12.122150          1
  199.000000         1
  56.333333          1
 Name: p_avg_talk_duration, Length: 767, dtype: int64,
 767,
 395.0,
 -1.0)

In [488]:
dfres, codes_p_avg_talk_duration = digital_one_hot_encoding(dfres, 'p_avg_talk_duration')
dfres = dfres.drop('p_avg_talk_duration', axis=1)

In [489]:
d = dfres['m_avg_duration'].value_counts()
l = len(d)
rmax = dfres['m_avg_duration'].max()
rmin = dfres['m_avg_duration'].min()
d, l, rmax, rmin

(-1.000       219858
  3.000          437
  5.000          428
  4.000          339
  6.000          299
               ...  
  286.750          1
  1549.527         1
  32.250           1
  185.250          1
  126.800          1
 Name: m_avg_duration, Length: 4873, dtype: int64,
 4873,
 3658.7811,
 -1.0)

In [490]:
dfres, codes_m_avg_duration = digital_one_hot_encoding(dfres, 'm_avg_duration')
dfres = dfres.drop('m_avg_duration', axis=1)

In [491]:
dfres, codes_p_avg_duration = digital_one_hot_encoding(dfres, 'p_avg_duration')
dfres = dfres.drop('p_avg_duration', axis=1)

In [492]:
d = dfres['m_missed_calls'].value_counts()
l = len(d)
rmax = dfres['m_missed_calls'].max()
rmin = dfres['m_missed_calls'].min()
d, l, rmax, rmin

(-1.0     211552
  0.0      44028
  1.0      13157
  2.0       6185
  3.0       5071
  4.0       2036
  5.0        991
  6.0        741
  7.0        346
  8.0        226
  11.0       191
  10.0        99
  9.0         94
  21.0        49
  13.0        44
  38.0        32
  14.0        19
  17.0        19
  18.0        17
  16.0        16
  26.0        15
  12.0        13
  15.0        12
  24.0        11
  19.0         9
  41.0         6
  33.0         6
  42.0         5
  20.0         4
  52.0         2
  22.0         1
 Name: m_missed_calls, dtype: int64,
 31,
 52.0,
 -1.0)

In [493]:
dfres, codes_m_missed_calls = digital_one_hot_encoding(dfres, 'm_missed_calls')
dfres = dfres.drop('m_missed_calls', axis=1)

In [494]:
d = dfres['p_missed_calls'].value_counts()
l = len(d)
rmax = dfres['p_missed_calls'].max()
rmin = dfres['p_missed_calls'].min()
d, l, rmax, rmin

(-1.0     261415
  0.0      15929
  1.0       4531
  2.0       1313
  3.0        526
  4.0        362
  5.0        285
  6.0        154
  7.0        133
  10.0       100
  8.0         83
  11.0        43
  58.0        32
  19.0        20
  9.0         18
  15.0        12
  17.0        12
  63.0        11
  16.0         9
  12.0         9
 Name: p_missed_calls, dtype: int64,
 20,
 63.0,
 -1.0)

In [495]:
dfres, codes_p_missed_calls = digital_one_hot_encoding(dfres, 'p_missed_calls')
dfres = dfres.drop('p_missed_calls', axis=1)

In [496]:
d = dfres['m_total_calls'].value_counts()
l = len(d)
rmax = dfres['m_total_calls'].max()
rmin = dfres['m_total_calls'].min()
d, l, rmax, rmin

(-1.0     211552
  1.0      24021
  2.0      16980
  3.0      14134
  4.0       6191
  5.0       3578
  6.0       2719
  7.0       1446
  8.0       1234
  9.0        784
  10.0       530
  11.0       446
  13.0       251
  12.0       184
  15.0       143
  14.0       127
  17.0       108
  16.0       105
  23.0        78
  21.0        74
  19.0        54
  18.0        48
  20.0        39
  22.0        28
  39.0        27
  24.0        21
  28.0        21
  52.0        15
  25.0         8
  32.0         7
  26.0         6
  42.0         6
  58.0         6
  40.0         5
  29.0         5
  33.0         5
  45.0         5
  80.0         2
  27.0         2
  36.0         1
  35.0         1
 Name: m_total_calls, dtype: int64,
 41,
 80.0,
 -1.0)

In [497]:
dfres, codes_m_total_calls = digital_one_hot_encoding(dfres, 'm_total_calls')
dfres = dfres.drop('m_total_calls', axis=1)

In [498]:
d = dfres['p_total_calls'].value_counts()
l = len(d)
rmax = dfres['p_total_calls'].max()
rmin = dfres['p_total_calls'].min()
d, l, rmax, rmin

(-1.0     261415
  1.0      12398
  2.0       6898
  3.0       1919
  4.0        795
  5.0        525
  6.0        350
  7.0        214
  8.0        135
  10.0        90
  11.0        52
  9.0         42
  12.0        32
  58.0        32
  19.0        20
  23.0        12
  18.0        12
  17.0        12
  63.0        11
  22.0        10
  44.0         9
  15.0         8
  20.0         4
  28.0         2
 Name: p_total_calls, dtype: int64,
 24,
 63.0,
 -1.0)

In [499]:
dfres, codes_p_total_calls = digital_one_hot_encoding(dfres, 'p_total_calls')
dfres = dfres.drop('p_total_calls', axis=1)

In [500]:
d = dfres['m_was_conversations'].value_counts()
l = len(d)
rmax = dfres['m_was_conversations'].max()
rmin = dfres['m_was_conversations'].min()
d, l, rmax, rmin

(-1.0     211552
  1.0      29458
  2.0      15773
  3.0       9823
  0.0       8306
  4.0       4048
  5.0       2150
  6.0       1420
  7.0        768
  8.0        568
  9.0        423
  10.0       215
  11.0       124
  12.0       100
  13.0        97
  15.0        40
  14.0        35
  26.0        15
  23.0        15
  20.0        14
  19.0        11
  17.0        10
  16.0         8
  25.0         6
  29.0         5
  21.0         5
  18.0         3
  28.0         2
  22.0         2
  32.0         1
 Name: m_was_conversations, dtype: int64,
 30,
 32.0,
 -1.0)

In [501]:
dfres, codes_m_was_conversations = digital_one_hot_encoding(dfres, 'm_was_conversations')
dfres = dfres.drop('m_was_conversations', axis=1)

In [502]:
d = dfres['p_was_conversations'].value_counts()
l = len(d)
rmax = dfres['p_was_conversations'].max()
rmin = dfres['p_was_conversations'].min()
d, l, rmax, rmin

(-1.0     261415
  1.0      11714
  2.0       6111
  0.0       4227
  3.0       1038
  4.0        287
  5.0        113
  7.0         27
  6.0         24
  19.0        10
  18.0        10
  28.0         9
  9.0          6
  8.0          4
  17.0         2
 Name: p_was_conversations, dtype: int64,
 15,
 28.0,
 -1.0)

In [503]:
dfres, codes_p_was_conversations = digital_one_hot_encoding(dfres, 'p_was_conversations')
dfres = dfres.drop('p_was_conversations', axis=1)

In [504]:
d = dfres['m_total_duration'].value_counts()
l = len(d)
rmax = dfres['m_total_duration'].max()
rmin = dfres['m_total_duration'].min()
d, l, rmax, rmin

(-1.000000       211552
  0.000000         8424
  7.000000          469
  10.000000         421
  9.000000          382
                  ...  
  263.103360          1
  1868.571429         1
  203.250000          1
  747.708267          1
  138.500000          1
 Name: m_total_duration, Length: 5057, dtype: int64,
 5057,
 7089.8766,
 -1.0)

In [505]:
dfres, codes_m_total_duration = digital_one_hot_encoding(dfres, 'm_total_duration')
dfres = dfres.drop('m_total_duration', axis=1)

In [506]:
d = dfres['p_total_duration'].value_counts()
l = len(d)
rmax = dfres['p_total_duration'].max()
rmin = dfres['p_total_duration'].min()
d, l, rmax, rmin

(-1.000000      261415
  0.000000        4270
  47.000000        278
  46.000000        278
  38.000000        268
                 ...  
  187.000000         1
  20.730633          1
  110.680500         1
  186.000000         1
  112.666667         1
 Name: p_total_duration, Length: 712, dtype: int64,
 712,
 395.0,
 -1.0)

In [507]:
dfres, codes_p_total_duration = digital_one_hot_encoding(dfres, 'p_total_duration')
dfres = dfres.drop('p_total_duration', axis=1)

In [508]:
d = dfres['support_feedback_avg'].value_counts()
l = len(d)
rmax = dfres['support_feedback_avg'].max()
rmin = dfres['support_feedback_avg'].min()
d, l, rmax, rmin

(-1.000000    231691
  5.000000     39237
  4.715000     10086
  4.000000      1066
  1.000000       476
               ...  
  4.569923         1
  4.100000         1
  4.818182         1
  2.714286         1
  4.659529         1
 Name: support_feedback_avg, Length: 93, dtype: int64,
 93,
 5.0,
 -1.0)

In [509]:
dfres, codes_support_feedback_avg = digital_one_hot_encoding(dfres, 'support_feedback_avg')
dfres = dfres.drop('support_feedback_avg', axis=1)

In [510]:
d = dfres['feedback_avg_d1'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d1'].max()
rmin = dfres['feedback_avg_d1'].min()
d, l, rmax, rmin

(-1.000000    215851
  5.000000     25965
  4.000000      7283
  4.715000      6032
  4.500000      2407
               ...  
  3.384615         1
  4.169053         1
  4.347826         1
  1.714286         1
  3.736842         1
 Name: feedback_avg_d1, Length: 1202, dtype: int64,
 1202,
 5.0,
 -1.0)

In [511]:
dfres, codes_feedback_avg_d1 = digital_one_hot_encoding(dfres, 'feedback_avg_d1')
dfres = dfres.drop('feedback_avg_d1', axis=1)

In [512]:
d = dfres['feedback_avg_d2'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d2'].max()
rmin = dfres['feedback_avg_d2'].min()
d, l, rmax, rmin

(-1.000000    263955
  5.000000      6268
  4.000000      2515
  4.715000      1744
  4.500000       698
               ...  
  3.894241         1
  3.526316         1
  4.409091         1
  3.562444         1
  3.611111         1
 Name: feedback_avg_d2, Length: 664, dtype: int64,
 664,
 5.0,
 -1.0)

In [513]:
dfres, codes_feedback_avg_d2 = digital_one_hot_encoding(dfres, 'feedback_avg_d2')
dfres = dfres.drop('feedback_avg_d2', axis=1)

In [514]:
d = dfres['feedback_avg_d3'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d3'].max()
rmin = dfres['feedback_avg_d3'].min()
d, l, rmax, rmin

(-1.000000    259579
  5.000000      9221
  4.000000      2452
  4.715000      1985
  4.500000       953
               ...  
  4.380387         1
  4.117647         1
  4.942308         1
  3.969697         1
  3.620690         1
 Name: feedback_avg_d3, Length: 847, dtype: int64,
 847,
 5.0,
 -1.0)

In [515]:
dfres, codes_feedback_avg_d3 = digital_one_hot_encoding(dfres, 'feedback_avg_d3')
dfres = dfres.drop('feedback_avg_d3', axis=1)

In [516]:
d = dfres['feedback_avg_d4'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d4'].max()
rmin = dfres['feedback_avg_d4'].min()
d, l, rmax, rmin

(-1.000000    249243
  5.000000     12643
  4.000000      3294
  4.715000      3064
  4.500000      1152
               ...  
  4.646418         1
  3.689281         1
  4.064516         1
  4.705755         1
  3.466667         1
 Name: feedback_avg_d4, Length: 1248, dtype: int64,
 1248,
 5.0,
 -1.0)

In [517]:
dfres, codes_feedback_avg_d4 = digital_one_hot_encoding(dfres, 'feedback_avg_d4')
dfres = dfres.drop('feedback_avg_d4', axis=1)

In [518]:
d = dfres['feedback_avg_d5'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d5'].max()
rmin = dfres['feedback_avg_d5'].min()
d, l, rmax, rmin

(-1.000000    262508
  5.000000      8204
  4.000000      2663
  4.715000      2170
  4.500000       639
               ...  
  3.047619         1
  4.657143         1
  4.649966         1
  4.160000         1
  4.562903         1
 Name: feedback_avg_d5, Length: 826, dtype: int64,
 826,
 5.0,
 -1.0)

In [519]:
dfres, codes_feedback_avg_d5 = digital_one_hot_encoding(dfres, 'feedback_avg_d5')
dfres = dfres.drop('feedback_avg_d5', axis=1)

In [520]:
dfres.to_csv("RSF_dfres.csv", index=True)

In [521]:
dfres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284997 entries, 0 to 284996
Columns: 1912 entries, id to feedback_avg_d55
dtypes: float64(1), int64(2), int8(1909)
memory usage: 525.4 MB


In [522]:
dfres, codes_target = simple_one_hot_encoding(dfres, 'target')

In [523]:
dfres = dfres.drop('target', axis=1)

In [524]:
dfres.head()

,id,student_id,age_indicator0,age_indicator1,age_indicator2,age_indicator3,age_indicator4,age_indicator5,age_indicator6,age_indicator7,...,feedback_avg_d52,feedback_avg_d53,feedback_avg_d54,feedback_avg_d55,target0,target1,target2,target3,target4,target5
0,15182,6694527,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,89385,6712877,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,47931,6659444,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,279085,7151591,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,7806,6705666,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


### Разбиваем обратно набор на обучающий и тестовый

In [525]:
def split_back(df, len_df):
    #features = df.iloc[:, 0:5] + df.iloc[:,11:]
    df1 = df.iloc[:len_df, :]
    df2 = df.iloc[len_df:, :]
    return df1, df2

In [526]:
data_df, test_df = split_back(dfres, len_df)

In [ ]:
def get_features_labels(df):
    features = df.iloc[:, 4:]
    labels = df.iloc[:, :4]
    return features, labels

In [ ]:
x_train, y_train = get_features_labels(train_df)

### Создаем нужные классы

In [ ]:
class BinaryDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        features = self.x[index, :]
        labels = self.y[index, :]

        # we have 12 feature columns 
        features = torch.tensor(features, dtype=torch.float32)
        # there are 5 classes and each class can have a binary value ...
        # ... either 0 or 1
        label1 = torch.tensor(labels[0], dtype=torch.float32)
        label2 = torch.tensor(labels[1], dtype=torch.float32)
        label3 = torch.tensor(labels[2], dtype=torch.float32)
        label4 = torch.tensor(labels[3], dtype=torch.float32)
        label5 = torch.tensor(labels[4], dtype=torch.float32)
        return {
            'features': features,
            'label1': label1,
            'label2': label2,
            'label3': label3,
            'label4': label4,
            'label5': label5,
        }

In [ ]:
class MultiHeadBinaryModel(nn.Module):
    def __init__(self):
        super(MultiHeadBinaryModel, self).__init__()
        self.fc1 = nn.Linear(12, 32) # 12 is the number of features
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, 256)

        # we will treat each head as a binary classifier ...
        # ... so the output features will be 1
        self.out1 = nn.Linear(256, 1)
        self.out2 = nn.Linear(256, 1)
        self.out3 = nn.Linear(256, 1)
        self.out4 = nn.Linear(256, 1)
        self.out5 = nn.Linear(256, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))

        # each binary classifier head will have its own output
        out1 = F.sigmoid(self.out1(x))
        out2 = F.sigmoid(self.out2(x))
        out3 = F.sigmoid(self.out3(x))
        out4 = F.sigmoid(self.out4(x))
        out5 = F.sigmoid(self.out5(x))

        return out1, out2, out3, out4, out5

In [ ]:
def binary_loss_fn(outputs, targets):
    o1, o2, o3, o4, o5 = outputs
    t1, t2, t3, t4, t5 = targets
    l1 = nn.BCELoss()(o1, t1)
    l2 = nn.BCELoss()(o2, t2)
    l3 = nn.BCELoss()(o3, t3)
    l4 = nn.BCELoss()(o4, t4)
    l5 = nn.BCELoss()(o5, t5)
    return (l1 + l2 + l3 + l4 + l5) / 5

In [ ]:
from dataset import make_dataset, BinaryDataset
from torch.utils.data import DataLoader
from loss_functions import binary_loss_fn as loss_fn
from models import MultiHeadBinaryModel
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

In [ ]:
#train_multi_head_binary.py
# prepare the dataset
x_train, y_train, _, _ = make_dataset()
# print some info
print(f"[INFO]: Number of training samples: {x_train.shape[0]}")
print(f"[INFO]: Number of training features: {x_train.shape[1]}")
# train dataset
train_dataset = BinaryDataset(x_train, y_train)
# train data loader
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=1024)
# initialize the model
model = MultiHeadBinaryModel()

In [ ]:
def train(model, dataloader, optimizer, loss_fn, train_dataset, device):
    model.train()
    counter = 0
    train_running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_dataset)/dataloader.batch_size)):
        counter += 1

        # extract the features and labels
        features = data['features'].to(device)
        target1 = data['label1'].to(device)
        target2 = data['label2'].to(device)
        target3 = data['label3'].to(device)
        target4 = data['label4'].to(device)
        target5 = data['label5'].to(device)

        # zero-out the optimizer gradients
        optimizer.zero_grad()

        outputs = model(features)
        targets = (target1, target2, target3, target4, target5)
        loss = loss_fn(outputs, targets)
        train_running_loss += loss.item()

        # backpropagation
        loss.backward()
        # update optimizer parameters
        optimizer.step()

    train_loss = train_running_loss / counter
    return train_loss

In [ ]:
optimizer = optim.Adam(params=model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 100
# load the model on to the computation device
model.to(device)

In [ ]:
#train_multi_head_binary.py
# start the training
train_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = train(
        model, train_dataloader, optimizer, loss_fn, train_dataset, device
    )
    train_loss.append(train_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
torch.save(model.state_dict(), 'outputs/multi_head_binary.pth')

In [ ]:
#train_multi_head_binary.py
# plot and save the train loss graph
plt.figure(figsize=(10, 7))
plt.plot(train_loss, color='orange', label='train loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('outputs/multi_head_binary_loss.png')
plt.show()

In [ ]:
#inference_multi_head_binary.py
from dataset import make_dataset, BinaryDataset
from torch.utils.data import DataLoader
from models import MultiHeadBinaryModel
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#inference_multi_head_binary.py
_, _, x_test, y_test = make_dataset()
# print some info
print(f"[INFO]: Number of test samples: {x_test.shape[0]}")
print(f"[INFO]: Number of test features: {x_test.shape[1]}")
test_dataset = BinaryDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1)

In [ ]:
#inference_multi_head_binary.py
# prepare the trained model
model = MultiHeadBinaryModel()
model.load_state_dict(torch.load('outputs/multi_head_binary.pth'))
model.to(device)
model.eval()

In [ ]:
#inference_multi_head_binary.py
for i, test_sample in enumerate(test_dataloader):
    print(f"SAMPLE {i}")
    # extract the features and labels
    features = test_sample['features'].to(device)
    target1 = test_sample['label1'].to(device)
    target2 = test_sample['label2'].to(device)
    target3 = test_sample['label3'].to(device)
    target4 = test_sample['label4'].to(device)
    target5 = test_sample['label5'].to(device)

    outputs = model(features)

    # get all the labels
    all_labels = []
    for out in outputs:
        if out >= 0.5:
            all_labels.append(1)
        else:
            all_labels.append(0)

    targets = (target1, target2, target3, target4, target5)

    # get all the targets in int format from tensor format
    all_targets = []
    for target in targets:
        all_targets.append(int(target.squeeze(0).detach().cpu()))

    print(f"ALL PREDICTIONS: {all_labels}")
    print(f"GROUND TRUTHS: {all_targets}")